# カタログ値とPhaseNet検測の比較サンプル

このノートブックでは、ObsPyを使用してサンプル地震波形データをダウンロードし、地震カタログの検測値とPhaseNetによる自動検測結果を可視化して比較します。

## セル 1: 必要なライブラリのインポート
まずは、ObsPyやプロット用のMatplotlibなどをインポートします。

In [ ]:
# ObsPyコア機能
import obspy
from obspy import UTCDateTime

# FDSNクライアント（波形データをダウンロードするため）
from obspy.clients.fdsn import Client

# グラフ描画ライブラリ
import matplotlib.pyplot as plt

print("ライブラリのインポートが完了しました。")

## セル 2: サンプル波形データの準備
ここでは例として、2011年の東北地方太平洋沖地震の波形データを、FDSNクライアント経由で取得します。

In [ ]:
# IRISのデータセンターに接続
client = Client("IRIS")

# 2011年東北地方太平洋沖地震の発生時刻
t = UTCDateTime("2011-03-11T05:46:23")

# ドイツの観測点(BFO)の波形データを取得 (地震発生から15分間)
# LHZ: 長周期上下動成分
st = client.get_waveforms("GR", "BFO", "00", "LHZ", t, t + 15 * 60)

# ノイズ除去のためバンドパスフィルターを適用
st.filter("bandpass", freqmin=0.01, freqmax=0.1)

# 扱いやすいように、Streamオブジェクトから最初のTraceオブジェクトを取り出す
tr = st[0]

print("波形データ:")
print(tr)

## セル 3: カタログ値とPhaseNet検測データの準備
実際にはカタログファイルやPhaseNetの出力結果から読み込むことになりますが、ここではサンプルとしてPythonの辞書型でデータを作成します。時刻は UTCDateTime で指定します。

In [ ]:
# 地震波形の開始時刻を基準にする
t0 = tr.stats.starttime

# カタログ値とPhaseNetの検測データを辞書で定義
# { '検測手法': {'P': P波時刻, 'S': S波時刻} }
detection_picks = {
    'Catalog': {
        'P': t0 + 715,
        'S': t0 + 820
    },
    'PhaseNet': {
        'P': t0 + 718,
        'S': t0 + 817
    }
}

# 色とスタイルの定義
pick_styles = {
    'Catalog': {
        'color': 'red',
        'linestyle': '-',
        'linewidth': 2
    },
    'PhaseNet': {
        'color': 'blue',
        'linestyle': '-',
        'linewidth': 2
    }
}

print("検測データの準備が完了しました。")
print("カタログ値:", detection_picks['Catalog'])
print("PhaseNet:", detection_picks['PhaseNet'])

## セル 4: 波形とカタログ値・PhaseNet検測位置をプロットして比較
波形の上に、カタログ値とPhaseNetの検測位置を色分けした縦線で描画して比較します。

In [ ]:
# グラフのサイズを指定
fig, ax = plt.subplots(figsize=(15, 6))

# 1. 波形をプロット
# x軸をmatplotlibの数値形式に変換してプロット
ax.plot(tr.times("matplotlib"), tr.data, "k-", label="Waveform", alpha=0.7)

# 2. カタログ値とPhaseNetの検測位置をプロット
for method, picks in detection_picks.items():
    style = pick_styles[method]
    
    # P波のピッキング位置に縦線を描画 (実線)
    p_time_mpl = picks['P'].matplotlib_date
    ax.axvline(p_time_mpl, 
               color=style['color'], 
               linestyle=style['linestyle'], 
               linewidth=style['linewidth'],
               label=f'{method} - P wave')
    
    # S波のピッキング位置に縦線を描画 (破線)
    s_time_mpl = picks['S'].matplotlib_date
    ax.axvline(s_time_mpl, 
               color=style['color'], 
               linestyle='--', 
               linewidth=style['linewidth'],
               label=f'{method} - S wave')

# 3. グラフの体裁を整える
ax.set_title("Comparison of Catalog vs PhaseNet Seismic Phase Picks", fontsize=16)
ax.set_xlabel(f"Time since {tr.stats.starttime.strftime('%Y-%m-%d %H:%M:%S')} [s]", fontsize=12)
ax.set_ylabel("Amplitude", fontsize=12)
ax.legend(loc='upper right') # 凡例を表示
ax.grid(True, alpha=0.3) # グリッド線を表示

# x軸のラベルをmatplotlibの日付形式から秒数に変換して見やすくする
from obspy.imaging.scripts.scan import _set_xaxis_obspy_datetimes
_set_xaxis_obspy_datetimes(ax, tr.stats.starttime)

# P波とS波の検測時差を計算して表示
catalog_ps_diff = detection_picks['Catalog']['S'] - detection_picks['Catalog']['P']
phasenet_ps_diff = detection_picks['PhaseNet']['S'] - detection_picks['PhaseNet']['P']

print(f"カタログ値 P-S時差: {catalog_ps_diff:.2f} 秒")
print(f"PhaseNet P-S時差: {phasenet_ps_diff:.2f} 秒")
print(f"P-S時差の差: {abs(catalog_ps_diff - phasenet_ps_diff):.2f} 秒")

plt.tight_layout()
plt.show()

## 今後の拡張計画

このノートブックの検測比較機能をベースに、以下のようなWeb GUIシステムの開発を計画しています。

### セル 5: Web GUIシステムの機能仕様

計画中のWeb GUIシステムは以下の機能を持つ予定です：

#### 主要機能
1. **波形表示とインタラクティブ編集**
   - リアルタイム波形表示
   - マウスクリックによるピッキング位置の修正
   - ズーム・パン機能による詳細確認

2. **PhaseNet結果の品質管理**
   - 確信度スコアによる検測結果のフィルタリング
   - 低確信度データの優先表示
   - バッチ処理による効率的な検証作業

3. **解析者支援機能**
   - 複数解析者による相互検証システム
   - 修正履歴の管理と追跡
   - 統計情報とレポート生成

### セル 6: システム拡張による利点

#### 1. 段階的導入による安全性
- **初期段階**: 人間による全数検証でリスクを最小化
- **中期段階**: 高確信度データの自動承認による効率化
- **長期段階**: PhaseNet精度向上に伴う完全自動化への移行

#### 2. 品質保証とトレーサビリティ
- **専門知識の活用**: 地震学者の経験を検測結果に反映
- **一貫性の確保**: 統一された基準による品質管理
- **修正履歴**: 全ての変更を記録し、監査可能な体制を構築

#### 3. 継続的改善サイクル
- **学習データの蓄積**: 修正結果をPhaseNetの再学習に活用
- **地域特性の反映**: 各観測網の特徴を学習モデルに組み込み
- **性能向上の可視化**: 精度改善の過程を定量的に評価

In [ ]:
# セル 7: システム拡張のためのサンプルコード構造

# 将来のWeb GUIシステムで使用予定の基本的なデータ構造例

class PhaseNetResult:
    """PhaseNet検測結果のデータクラス"""
    def __init__(self, station, time, phase, confidence, is_verified=False):
        self.station = station
        self.time = time
        self.phase = phase  # 'P' or 'S'
        self.confidence = confidence  # 0.0 - 1.0
        self.is_verified = is_verified
        self.modified_by = None
        self.modified_time = None

class VerificationSession:
    """検証セッションの管理クラス"""
    def __init__(self, analyst_id):
        self.analyst_id = analyst_id
        self.results = []
        self.statistics = {
            'total_reviewed': 0,
            'modifications_made': 0,
            'accuracy_improvement': 0.0
        }
    
    def verify_pick(self, result, new_time=None, approved=True):
        """検測結果の検証・修正"""
        if new_time:
            result.time = new_time
            self.statistics['modifications_made'] += 1
        result.is_verified = approved
        result.modified_by = self.analyst_id
        result.modified_time = UTCDateTime.now()
        self.statistics['total_reviewed'] += 1

# サンプルデータの作成
sample_phasenet_results = [
    PhaseNetResult('BFO', t0 + 718, 'P', 0.95),
    PhaseNetResult('BFO', t0 + 817, 'S', 0.87),
]

print("Web GUIシステム用のデータ構造例を定義しました。")
print(f"PhaseNet P波検測: 時刻={sample_phasenet_results[0].time}, 確信度={sample_phasenet_results[0].confidence}")
print(f"PhaseNet S波検測: 時刻={sample_phasenet_results[1].time}, 確信度={sample_phasenet_results[1].confidence}")